In [1]:
import numpy as np
from scipy import spatial

def euclidian_distance(v1, v2):
    dist = np.linalg.norm(v1-v2)
    return dist

def cosine_distance(v1, v2):
    dist = spatial.distance.cosine(v1, v2)
    return dist

def cosine_distance2(v1, v2):
    dist = 1 - np.dot(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))
    return dist

def get_neighbors(k, v, data, dist_function = 'euclidian'):
    """
    k : the number of neighbors
    v : a vector looking for k neigbor vectors
    data : a matrix of data contains rows of potential neighbors
    """
    distances = list()
    for row in data:
        if dist_function == 'euclidian':
            dist = euclidian_distance(v, row)
        elif dist_function == 'cosine':
            dist = cosine_distance(v, row)
        distances.append((row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(k):
        neighbors.append(distances[i][0])
    return neighbors

In [2]:
dataset = [[2.7810836,2.550537003,0],
           [1.465489372,2.362125076,0],
           [3.396561688,4.400293529,0],
           [1.38807019,1.850220317,0],
           [3.06407232,3.005305973,0],
           [7.627531214,2.759262235,1],
           [5.332441248,2.088626775,1],
           [6.922596716,1.77106367,1],
           [8.675418651,-0.242068655,1],
           [7.673756466,3.508563011,1]]
dataset = np.array(dataset)
dataset

array([[ 2.7810836 ,  2.550537  ,  0.        ],
       [ 1.46548937,  2.36212508,  0.        ],
       [ 3.39656169,  4.40029353,  0.        ],
       [ 1.38807019,  1.85022032,  0.        ],
       [ 3.06407232,  3.00530597,  0.        ],
       [ 7.62753121,  2.75926224,  1.        ],
       [ 5.33244125,  2.08862677,  1.        ],
       [ 6.92259672,  1.77106367,  1.        ],
       [ 8.67541865, -0.24206865,  1.        ],
       [ 7.67375647,  3.50856301,  1.        ]])

In [3]:
get_neighbors(3, dataset[0], dataset, dist_function = 'euclidian')

[array([2.7810836, 2.550537 , 0.       ]),
 array([3.06407232, 3.00530597, 0.        ]),
 array([1.46548937, 2.36212508, 0.        ])]